<h1>Running Regression Models for MLP + LSTM</h1>

In [1]:
import os
import time

In [2]:
os.getcwd()

'C:\\Users\\13473\\Desktop\\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664\\Ipynbs To Run'

In [3]:
wd_path = r'../'
os.chdir(wd_path)
wd_path=os.getcwd()

In [4]:
import pandas as pd
import numpy as np

from datetime import datetime

In [5]:
normalized_data = 'complete_universe_cleaned_normalized_jonn.csv'
auto_encoder_data = "Autoencoder_transformed_43_dim_data.csv"

In [6]:
start_time = time.time()
df = pd.read_csv(normalized_data ,dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')
df = df.reset_index()
print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


--- 45.72796845436096 seconds ---


In [7]:
start_time = time.time()
auto = pd.read_csv(auto_encoder_data,low_memory=False,index_col=0)
print("--- %s seconds ---" % (time.time() - start_time))

--- 20.194296836853027 seconds ---


In [8]:
df.columns

Index(['index', 'DATE', 'COMNAM', 'NCUSIP', 'PERMNO', 'PERMCO', 'SICCD',
       'TICKER_PERMNO', 'accru', 'adv_sale', 'aftret_eq', 'aftret_equity',
       'aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat',
       'debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov',
       'intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat',
       'pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity',
       'sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap', 'm-12',
       'm-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2',
       'm-1', 'm+1'],
      dtype='object')

In [9]:
identifier_cols = ['DATE','COMNAM','NCUSIP','PERMNO','PERMCO','SICCD','TICKER_PERMNO']
cross_sectional_cols=['accru', 'adv_sale', 'aftret_eq', 'aftret_equity','aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat','debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov','intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat','pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity','sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap']
sector_cols = ['Agriculture, Forestry and Fishing', 'Construction','Finance, Insurance', 'Manufacturing', 'Mining', 'Nonclassifiable',
       'Public Administration', 'Real Estate', 'Retail Trade', 'Services','Transportation, Communications, Electric, Gas and Sanitary service',
       'Wholesale Trade', 'not used']
time_series_cols= ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1']
Y = ['m+1']
returns =  ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1','m+1']

In [10]:
new_df = pd.concat([df[identifier_cols],auto,df[returns]], axis=1, sort=False)

In [11]:
del df
del auto 
df = None
auto = None

In [12]:
new_df.columns

Index(['DATE', 'COMNAM', 'NCUSIP', 'PERMNO', 'PERMCO', 'SICCD',
       'TICKER_PERMNO', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', 'm-12', 'm-11', 'm-10',
       'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2', 'm-1', 'm+1'],
      dtype='object')

In [13]:
cross_sectional_cols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42']
time_series_cols = ['m-12', 'm-11', 'm-10',
       'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2', 'm-1']
Y = ['m+1']

In [14]:
len(cross_sectional_cols)

43

In [15]:
len(time_series_cols)

12

In [16]:
new_df.loc[:,'m+1'] = new_df.groupby(['DATE'])['m+1'].transform(lambda x: x.rank(pct = True))

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

In [18]:
'''Model 4: LSTM + MLP Regression'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(43,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model2.compile(
    optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
)

In [19]:
hybrid_model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 1)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 12, 50)       10400       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 43)]         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 30)           9720        lstm[0][0]                       
_______________________________________________________________________________________

In [20]:
dates = new_df['DATE'].unique()

In [21]:
dates

array(['1971-01-31T00:00:00.000000000', '1971-02-28T00:00:00.000000000',
       '1971-03-31T00:00:00.000000000', '1971-04-30T00:00:00.000000000',
       '1971-05-31T00:00:00.000000000', '1971-06-30T00:00:00.000000000',
       '1971-07-31T00:00:00.000000000', '1971-08-31T00:00:00.000000000',
       '1971-09-30T00:00:00.000000000', '1971-10-31T00:00:00.000000000',
       '1971-11-30T00:00:00.000000000', '1971-12-31T00:00:00.000000000',
       '1972-01-31T00:00:00.000000000', '1972-02-29T00:00:00.000000000',
       '1972-03-31T00:00:00.000000000', '1972-04-30T00:00:00.000000000',
       '1972-05-31T00:00:00.000000000', '1972-06-30T00:00:00.000000000',
       '1972-07-31T00:00:00.000000000', '1972-08-31T00:00:00.000000000',
       '1972-09-30T00:00:00.000000000', '1972-10-31T00:00:00.000000000',
       '1972-11-30T00:00:00.000000000', '1972-12-31T00:00:00.000000000',
       '1973-01-31T00:00:00.000000000', '1973-02-28T00:00:00.000000000',
       '1973-03-31T00:00:00.000000000', '1973-04-30

In [22]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [23]:
import gc

In [25]:
start_time = time.time()

x = [new_df[cross_sectional_cols].values,new_df[time_series_cols].values]
y = new_df[Y].values[:,0]

for dt in dates[119:]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    print(start_date,end_date)
    
    mask_train=(new_df[['DATE']]>=start_date) & (new_df[['DATE']]<=end_date)
    
    x_train, y_train=[i[mask_train['DATE'],:] for i in x], y[mask_train['DATE']]
    
    hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model2.compile(
        optimizer=opt, loss='binary_crossentropy')
    
    mcp_save = ModelCheckpoint(os.path.join(wd_path,'hdf5_files\hybrid_reg_model_w_auto_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'), save_best_only=True, monitor='val_loss', mode='min') 
    
    history_hybrid2=hybrid_model2.fit(
        x=x_train, y=y_train, batch_size=512, epochs=2, verbose=1, 
        callbacks=[mcp_save],
        validation_split=0.1
    )
    
    del hybrid_model2
    del history_hybrid2
    del x_train
    del y_train
    gc.collect()
    hybrid_model2 = None
    history_hybrid2 = None
    x_train = None
    y_train = None
print("--- %s seconds ---" % (time.time() - start_time))

1970-12-31 00:00:00 1980-12-31 00:00:00
Epoch 1/2
607/607 [==============================] - 4s 7ms/step - loss: 0.6852 - val_loss: 0.6946
Epoch 2/2
607/607 [==============================] - 4s 6ms/step - loss: 0.6847 - val_loss: 0.6956
1971-01-31 00:00:00 1981-01-31 00:00:00
Epoch 1/2
613/613 [==============================] - 4s 7ms/step - loss: 0.6841 - val_loss: 0.6967
Epoch 2/2
613/613 [==============================] - 4s 6ms/step - loss: 0.6836 - val_loss: 0.6966
1971-02-28 00:00:00 1981-02-28 00:00:00
Epoch 1/2
615/615 [==============================] - 4s 7ms/step - loss: 0.6829 - val_loss: 0.6977
Epoch 2/2
615/615 [==============================] - 4s 6ms/step - loss: 0.6823 - val_loss: 0.7009
1971-03-31 00:00:00 1981-03-31 00:00:00
Epoch 1/2
618/618 [==============================] - 4s 7ms/step - loss: 0.6817 - val_loss: 0.7013
Epoch 2/2
618/618 [==============================] - 4s 6ms/step - loss: 0.6812 - val_loss: 0.7033
1971-04-30 00:00:00 1981-04-30 00:00:00
Epoch 1/

760/760 [==============================] - 5s 6ms/step - loss: 0.6722 - val_loss: 0.7030
1976-09-30 00:00:00 1986-09-30 00:00:00
Epoch 1/2
762/762 [==============================] - 5s 7ms/step - loss: 0.6725 - val_loss: 0.7006
Epoch 2/2
762/762 [==============================] - 5s 6ms/step - loss: 0.6723 - val_loss: 0.7023
1976-10-31 00:00:00 1986-10-31 00:00:00
Epoch 1/2
763/763 [==============================] - 5s 7ms/step - loss: 0.6721 - val_loss: 0.6981
Epoch 2/2
763/763 [==============================] - 5s 6ms/step - loss: 0.6721 - val_loss: 0.7038
1976-11-30 00:00:00 1986-11-30 00:00:00
Epoch 1/2
765/765 [==============================] - 5s 7ms/step - loss: 0.6726 - val_loss: 0.6989
Epoch 2/2
765/765 [==============================] - 5s 6ms/step - loss: 0.6722 - val_loss: 0.7021
1976-12-31 00:00:00 1986-12-31 00:00:00
Epoch 1/2
767/767 [==============================] - 5s 7ms/step - loss: 0.6723 - val_loss: 0.7005
Epoch 2/2
767/767 [==============================] - 5s 6m

873/873 [==============================] - 6s 7ms/step - loss: 0.6758 - val_loss: 0.7035
Epoch 2/2
873/873 [==============================] - 5s 6ms/step - loss: 0.6754 - val_loss: 0.7030
1982-06-30 00:00:00 1992-06-30 00:00:00
Epoch 1/2
874/874 [==============================] - 6s 7ms/step - loss: 0.6756 - val_loss: 0.7014
Epoch 2/2
874/874 [==============================] - 6s 6ms/step - loss: 0.6757 - val_loss: 0.7059
1982-07-31 00:00:00 1992-07-31 00:00:00
Epoch 1/2
875/875 [==============================] - 6s 7ms/step - loss: 0.6759 - val_loss: 0.7079
Epoch 2/2
875/875 [==============================] - 5s 6ms/step - loss: 0.6755 - val_loss: 0.7042
1982-08-31 00:00:00 1992-08-31 00:00:00
Epoch 1/2
876/876 [==============================] - 6s 7ms/step - loss: 0.6757 - val_loss: 0.7058
Epoch 2/2
876/876 [==============================] - 5s 6ms/step - loss: 0.6757 - val_loss: 0.7019
1982-09-30 00:00:00 1992-09-30 00:00:00
Epoch 1/2
877/877 [==============================] - 6s 7m

1036/1036 [==============================] - 7s 6ms/step - loss: 0.6792 - val_loss: 0.6988
1988-02-28 00:00:00 1998-02-28 00:00:00
Epoch 1/2
1039/1039 [==============================] - 7s 7ms/step - loss: 0.6792 - val_loss: 0.6959
Epoch 2/2
1039/1039 [==============================] - 7s 6ms/step - loss: 0.6792 - val_loss: 0.7002
1988-03-31 00:00:00 1998-03-31 00:00:00
Epoch 1/2
1042/1042 [==============================] - 7s 7ms/step - loss: 0.6796 - val_loss: 0.6976
Epoch 2/2
1042/1042 [==============================] - 7s 6ms/step - loss: 0.6793 - val_loss: 0.6991
1988-04-30 00:00:00 1998-04-30 00:00:00
Epoch 1/2
1046/1046 [==============================] - 7s 7ms/step - loss: 0.6795 - val_loss: 0.6995
Epoch 2/2
1046/1046 [==============================] - 7s 6ms/step - loss: 0.6794 - val_loss: 0.7035
1988-05-31 00:00:00 1998-05-31 00:00:00
Epoch 1/2
1049/1049 [==============================] - 7s 7ms/step - loss: 0.6795 - val_loss: 0.6976
Epoch 2/2
1049/1049 [=====================

1993-09-30 00:00:00 2003-09-30 00:00:00
Epoch 1/2
1177/1177 [==============================] - 8s 7ms/step - loss: 0.6787 - val_loss: 0.7033
Epoch 2/2
1177/1177 [==============================] - 7s 6ms/step - loss: 0.6788 - val_loss: 0.7056
1993-10-31 00:00:00 2003-10-31 00:00:00
Epoch 1/2
1177/1177 [==============================] - 8s 7ms/step - loss: 0.6785 - val_loss: 0.7073
Epoch 2/2
1177/1177 [==============================] - 7s 6ms/step - loss: 0.6786 - val_loss: 0.7075
1993-11-30 00:00:00 2003-11-30 00:00:00
Epoch 1/2
1177/1177 [==============================] - 8s 7ms/step - loss: 0.6789 - val_loss: 0.7068
Epoch 2/2
1177/1177 [==============================] - 7s 6ms/step - loss: 0.6787 - val_loss: 0.7061
1993-12-31 00:00:00 2003-12-31 00:00:00
Epoch 1/2
1176/1176 [==============================] - 8s 7ms/step - loss: 0.6788 - val_loss: 0.7078
Epoch 2/2
1176/1176 [==============================] - 7s 6ms/step - loss: 0.6787 - val_loss: 0.7061
1994-01-31 00:00:00 2004-01-31 0

Epoch 2/2
993/993 [==============================] - 6s 6ms/step - loss: 0.6786 - val_loss: 0.7104
1999-05-31 00:00:00 2009-05-31 00:00:00
Epoch 1/2
990/990 [==============================] - 7s 7ms/step - loss: 0.6787 - val_loss: 0.7121
Epoch 2/2
990/990 [==============================] - 6s 6ms/step - loss: 0.6784 - val_loss: 0.7110
1999-06-30 00:00:00 2009-06-30 00:00:00
Epoch 1/2
986/986 [==============================] - 7s 7ms/step - loss: 0.6785 - val_loss: 0.7113
Epoch 2/2
986/986 [==============================] - 6s 6ms/step - loss: 0.6786 - val_loss: 0.7130
1999-07-31 00:00:00 2009-07-31 00:00:00
Epoch 1/2
983/983 [==============================] - 7s 7ms/step - loss: 0.6786 - val_loss: 0.7141
Epoch 2/2
983/983 [==============================] - 6s 6ms/step - loss: 0.6785 - val_loss: 0.7088
1999-08-31 00:00:00 2009-08-31 00:00:00
Epoch 1/2
979/979 [==============================] - 7s 7ms/step - loss: 0.6786 - val_loss: 0.7091
Epoch 2/2
979/979 [=============================

803/803 [==============================] - 6s 7ms/step - loss: 0.6772 - val_loss: 0.7105
Epoch 2/2
803/803 [==============================] - 5s 6ms/step - loss: 0.6770 - val_loss: 0.7115
2005-02-28 00:00:00 2015-02-28 00:00:00
Epoch 1/2
801/801 [==============================] - 6s 7ms/step - loss: 0.6771 - val_loss: 0.7125
Epoch 2/2
801/801 [==============================] - 5s 6ms/step - loss: 0.6772 - val_loss: 0.7106
2005-03-31 00:00:00 2015-03-31 00:00:00
Epoch 1/2
800/800 [==============================] - 6s 7ms/step - loss: 0.6774 - val_loss: 0.7144
Epoch 2/2
800/800 [==============================] - 5s 6ms/step - loss: 0.6771 - val_loss: 0.7099
2005-04-30 00:00:00 2015-04-30 00:00:00
Epoch 1/2
798/798 [==============================] - 6s 7ms/step - loss: 0.6773 - val_loss: 0.7130
Epoch 2/2
798/798 [==============================] - 5s 6ms/step - loss: 0.6773 - val_loss: 0.7087
2005-05-31 00:00:00 2015-05-31 00:00:00
Epoch 1/2
796/796 [==============================] - 6s 7m

In [24]:
'''Model 3: LSTM + MLP Classification + AutoEncoders'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(43,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(5, activation='softmax')(h3)

hybrid_model1=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model1.compile(
    optimizer=opt, loss='CategoricalCrossentropy',metrics=['accuracy']
)


In [25]:
start_time = time.time()

x = [new_df[cross_sectional_cols].values,new_df[time_series_cols].values]
y = pd.get_dummies(pd.qcut(new_df[Y].values[:,0], 5, range(0,5))).values 

for dt in dates[312:]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    print(start_date,end_date)
    
    mask_train=(new_df[['DATE']]>=start_date) & (new_df[['DATE']]<=end_date)
    
    x_train, y_train=[i[mask_train['DATE'],:] for i in x], y[mask_train['DATE']]
    
    hybrid_model1=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model1.compile(
        optimizer=opt, loss='CategoricalCrossentropy',metrics=['accuracy']
    )
    
    mcp_save = ModelCheckpoint(os.path.join(wd_path,'hdf5_files\hybrid_class_model_w_auto_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'), save_best_only=True, monitor='val_loss', mode='min') 
    
    history_hybrid1=hybrid_model1.fit(
        x=x_train, y=y_train, batch_size=512, epochs=2, verbose=1, 
        callbacks=[mcp_save],
        validation_split=0.1
    )
    
    del hybrid_model1
    del history_hybrid1
    del x_train
    del y_train
    gc.collect()
    hybrid_model1 = None
    history_hybrid1 = None
    x_train = None
    y_train = None
print("--- %s seconds ---" % (time.time() - start_time))

1987-01-31 00:00:00 1997-01-31 00:00:00
Epoch 1/2
995/995 [==============================] - 7s 7ms/step - loss: 1.5501 - accuracy: 0.2793 - val_loss: 1.5453 - val_accuracy: 0.2883
Epoch 2/2
995/995 [==============================] - 6s 7ms/step - loss: 1.5432 - accuracy: 0.2859 - val_loss: 1.5335 - val_accuracy: 0.2958
1987-02-28 00:00:00 1997-02-28 00:00:00
Epoch 1/2
998/998 [==============================] - 7s 7ms/step - loss: 1.5414 - accuracy: 0.2874 - val_loss: 1.5418 - val_accuracy: 0.2924
Epoch 2/2
998/998 [==============================] - 6s 6ms/step - loss: 1.5394 - accuracy: 0.2886 - val_loss: 1.5425 - val_accuracy: 0.2945
1987-03-31 00:00:00 1997-03-31 00:00:00
Epoch 1/2
1002/1002 [==============================] - 7s 7ms/step - loss: 1.5383 - accuracy: 0.2904 - val_loss: 1.5384 - val_accuracy: 0.3004
Epoch 2/2
1002/1002 [==============================] - 6s 6ms/step - loss: 1.5368 - accuracy: 0.2914 - val_loss: 1.5394 - val_accuracy: 0.2962
1987-04-30 00:00:00 1997-04-30

1076/1076 [==============================] - 7s 7ms/step - loss: 1.5073 - accuracy: 0.3138 - val_loss: 1.5715 - val_accuracy: 0.2675
Epoch 2/2
1076/1076 [==============================] - 7s 6ms/step - loss: 1.5061 - accuracy: 0.3148 - val_loss: 1.5797 - val_accuracy: 0.2700
1989-03-31 00:00:00 1999-03-31 00:00:00
Epoch 1/2
1079/1079 [==============================] - 7s 7ms/step - loss: 1.5063 - accuracy: 0.3146 - val_loss: 1.5757 - val_accuracy: 0.2681
Epoch 2/2
1079/1079 [==============================] - 7s 6ms/step - loss: 1.5063 - accuracy: 0.3150 - val_loss: 1.5793 - val_accuracy: 0.2674
1989-04-30 00:00:00 1999-04-30 00:00:00
Epoch 1/2
1082/1082 [==============================] - 7s 7ms/step - loss: 1.5067 - accuracy: 0.3141 - val_loss: 1.5799 - val_accuracy: 0.2675
Epoch 2/2
1082/1082 [==============================] - 7s 6ms/step - loss: 1.5060 - accuracy: 0.3145 - val_loss: 1.5768 - val_accuracy: 0.2709
1989-05-31 00:00:00 1999-05-31 00:00:00
Epoch 1/2
1085/1085 [===========

1144/1144 [==============================] - 8s 7ms/step - loss: 1.5092 - accuracy: 0.3120 - val_loss: 1.5734 - val_accuracy: 0.2761
Epoch 2/2
1144/1144 [==============================] - 8s 7ms/step - loss: 1.5083 - accuracy: 0.3127 - val_loss: 1.5694 - val_accuracy: 0.2796
1991-04-30 00:00:00 2001-04-30 00:00:00
Epoch 1/2
1146/1146 [==============================] - 8s 7ms/step - loss: 1.5091 - accuracy: 0.3119 - val_loss: 1.5648 - val_accuracy: 0.2772
Epoch 2/2
1146/1146 [==============================] - 8s 7ms/step - loss: 1.5085 - accuracy: 0.3124 - val_loss: 1.5690 - val_accuracy: 0.2791
1991-05-31 00:00:00 2001-05-31 00:00:00
Epoch 1/2
1149/1149 [==============================] - 8s 7ms/step - loss: 1.5092 - accuracy: 0.3115 - val_loss: 1.5677 - val_accuracy: 0.2826
Epoch 2/2
1149/1149 [==============================] - 8s 7ms/step - loss: 1.5086 - accuracy: 0.3122 - val_loss: 1.5622 - val_accuracy: 0.2837
1991-06-30 00:00:00 2001-06-30 00:00:00
Epoch 1/2
1151/1151 [===========

1177/1177 [==============================] - 8s 7ms/step - loss: 1.5096 - accuracy: 0.3125 - val_loss: 1.5581 - val_accuracy: 0.2821
Epoch 2/2
1177/1177 [==============================] - 8s 7ms/step - loss: 1.5081 - accuracy: 0.3128 - val_loss: 1.5598 - val_accuracy: 0.2844
1993-05-31 00:00:00 2003-05-31 00:00:00
Epoch 1/2
1177/1177 [==============================] - 8s 7ms/step - loss: 1.5085 - accuracy: 0.3132 - val_loss: 1.5613 - val_accuracy: 0.2811
Epoch 2/2
1177/1177 [==============================] - 8s 7ms/step - loss: 1.5073 - accuracy: 0.3130 - val_loss: 1.5696 - val_accuracy: 0.2792
1993-06-30 00:00:00 2003-06-30 00:00:00
Epoch 1/2
1177/1177 [==============================] - 8s 7ms/step - loss: 1.5092 - accuracy: 0.3130 - val_loss: 1.5714 - val_accuracy: 0.2745
Epoch 2/2
1177/1177 [==============================] - 8s 6ms/step - loss: 1.5074 - accuracy: 0.3136 - val_loss: 1.5700 - val_accuracy: 0.2744
1993-07-31 00:00:00 2003-07-31 00:00:00
Epoch 1/2
1177/1177 [===========

1150/1150 [==============================] - 8s 7ms/step - loss: 1.5076 - accuracy: 0.3114 - val_loss: 1.5969 - val_accuracy: 0.2773
Epoch 2/2
1150/1150 [==============================] - 7s 6ms/step - loss: 1.5068 - accuracy: 0.3121 - val_loss: 1.5825 - val_accuracy: 0.2772
1995-06-30 00:00:00 2005-06-30 00:00:00
Epoch 1/2
1148/1148 [==============================] - 8s 7ms/step - loss: 1.5075 - accuracy: 0.3116 - val_loss: 1.5793 - val_accuracy: 0.2772
Epoch 2/2
1148/1148 [==============================] - 7s 6ms/step - loss: 1.5070 - accuracy: 0.3124 - val_loss: 1.5824 - val_accuracy: 0.2774
1995-07-31 00:00:00 2005-07-31 00:00:00
Epoch 1/2
1146/1146 [==============================] - 8s 7ms/step - loss: 1.5080 - accuracy: 0.3117 - val_loss: 1.5842 - val_accuracy: 0.2757
Epoch 2/2
1146/1146 [==============================] - 7s 6ms/step - loss: 1.5066 - accuracy: 0.3117 - val_loss: 1.5806 - val_accuracy: 0.2764
1995-08-31 00:00:00 2005-08-31 00:00:00
Epoch 1/2
1143/1143 [===========

...

983/983 [==============================] - 7s 7ms/step - loss: 1.5116 - accuracy: 0.3064 - val_loss: 1.6181 - val_accuracy: 0.2462
Epoch 2/2
983/983 [==============================] - 6s 6ms/step - loss: 1.5097 - accuracy: 0.3078 - val_loss: 1.6279 - val_accuracy: 0.2466
1999-08-31 00:00:00 2009-08-31 00:00:00
Epoch 1/2
979/979 [==============================] - 7s 7ms/step - loss: 1.5117 - accuracy: 0.3063 - val_loss: 1.6140 - val_accuracy: 0.2459
Epoch 2/2
979/979 [==============================] - 6s 6ms/step - loss: 1.5120 - accuracy: 0.3071 - val_loss: 1.6380 - val_accuracy: 0.2529
1999-09-30 00:00:00 2009-09-30 00:00:00
Epoch 1/2
976/976 [==============================] - 7s 7ms/step - loss: 1.5126 - accuracy: 0.3070 - val_loss: 1.6114 - val_accuracy: 0.2516
Epoch 2/2
976/976 [==============================] - 6s 6ms/step - loss: 1.5098 - accuracy: 0.3079 - val_loss: 1.6276 - val_accuracy: 0.2474
1999-10-31 00:00:00 2009-10-31 00:00:00
Epoch 1/2
972/972 [=========================

895/895 [==============================] - 6s 6ms/step - loss: 1.5163 - accuracy: 0.3016 - val_loss: 1.5974 - val_accuracy: 0.2660
2001-09-30 00:00:00 2011-09-30 00:00:00
Epoch 1/2
892/892 [==============================] - 6s 7ms/step - loss: 1.5160 - accuracy: 0.3020 - val_loss: 1.5995 - val_accuracy: 0.2689
Epoch 2/2
892/892 [==============================] - 5s 6ms/step - loss: 1.5181 - accuracy: 0.3006 - val_loss: 1.5822 - val_accuracy: 0.2626
2001-10-31 00:00:00 2011-10-31 00:00:00
Epoch 1/2
889/889 [==============================] - 6s 7ms/step - loss: 1.5180 - accuracy: 0.3001 - val_loss: 1.5853 - val_accuracy: 0.2697
Epoch 2/2
889/889 [==============================] - 5s 6ms/step - loss: 1.5149 - accuracy: 0.3020 - val_loss: 1.6016 - val_accuracy: 0.2604
2001-11-30 00:00:00 2011-11-30 00:00:00
Epoch 1/2
886/886 [==============================] - 6s 7ms/step - loss: 1.5168 - accuracy: 0.3006 - val_loss: 1.5877 - val_accuracy: 0.2632
Epoch 2/2
886/886 [=========================

2003-10-31 00:00:00 2013-10-31 00:00:00
Epoch 1/2
830/830 [==============================] - 6s 7ms/step - loss: 1.5229 - accuracy: 0.2981 - val_loss: 1.5966 - val_accuracy: 0.2749
Epoch 2/2
830/830 [==============================] - 5s 6ms/step - loss: 1.5213 - accuracy: 0.2985 - val_loss: 1.5893 - val_accuracy: 0.2715
2003-11-30 00:00:00 2013-11-30 00:00:00
Epoch 1/2
828/828 [==============================] - 5s 7ms/step - loss: 1.5203 - accuracy: 0.2989 - val_loss: 1.6043 - val_accuracy: 0.2739
Epoch 2/2
828/828 [==============================] - 5s 6ms/step - loss: 1.5170 - accuracy: 0.3003 - val_loss: 1.5986 - val_accuracy: 0.2723
2003-12-31 00:00:00 2013-12-31 00:00:00
Epoch 1/2
826/826 [==============================] - 6s 7ms/step - loss: 1.5202 - accuracy: 0.2994 - val_loss: 1.6077 - val_accuracy: 0.2693
Epoch 2/2
826/826 [==============================] - 5s 6ms/step - loss: 1.5174 - accuracy: 0.3003 - val_loss: 1.6216 - val_accuracy: 0.2676
2004-01-31 00:00:00 2014-01-31 00:

Epoch 2/2
787/787 [==============================] - 5s 6ms/step - loss: 1.5301 - accuracy: 0.2920 - val_loss: 1.6311 - val_accuracy: 0.2830
2005-12-31 00:00:00 2015-12-31 00:00:00
Epoch 1/2
785/785 [==============================] - 6s 7ms/step - loss: 1.5238 - accuracy: 0.2968 - val_loss: 1.7556 - val_accuracy: 0.2804
Epoch 2/2
785/785 [==============================] - 5s 7ms/step - loss: 1.5212 - accuracy: 0.2985 - val_loss: 1.6663 - val_accuracy: 0.2810
2006-01-31 00:00:00 2016-01-31 00:00:00
Epoch 1/2
784/784 [==============================] - 6s 7ms/step - loss: 1.5204 - accuracy: 0.2987 - val_loss: 1.7201 - val_accuracy: 0.2770
Epoch 2/2
784/784 [==============================] - 5s 6ms/step - loss: 1.5229 - accuracy: 0.2971 - val_loss: 1.7401 - val_accuracy: 0.2803
2006-02-28 00:00:00 2016-02-29 00:00:00
Epoch 1/2
782/782 [==============================] - 5s 7ms/step - loss: 1.5250 - accuracy: 0.2957 - val_loss: 1.7130 - val_accuracy: 0.2821
Epoch 2/2
782/782 [===============

2008-01-31 00:00:00 2018-01-31 00:00:00
Epoch 1/2
746/746 [==============================] - 5s 7ms/step - loss: 1.5174 - accuracy: 0.2999 - val_loss: 1.5702 - val_accuracy: 0.2798
Epoch 2/2
746/746 [==============================] - 5s 6ms/step - loss: 1.5150 - accuracy: 0.3008 - val_loss: 1.5699 - val_accuracy: 0.2808
2008-02-28 00:00:00 2018-02-28 00:00:00
Epoch 1/2
745/745 [==============================] - 5s 7ms/step - loss: 1.5133 - accuracy: 0.3020 - val_loss: 1.5758 - val_accuracy: 0.2845
Epoch 2/2
745/745 [==============================] - 5s 6ms/step - loss: 1.5183 - accuracy: 0.2987 - val_loss: 1.5639 - val_accuracy: 0.2799
2008-03-31 00:00:00 2018-03-31 00:00:00
Epoch 1/2
744/744 [==============================] - 5s 7ms/step - loss: 1.5238 - accuracy: 0.2984 - val_loss: 1.5713 - val_accuracy: 0.2822
Epoch 2/2
744/744 [==============================] - 5s 6ms/step - loss: 1.5144 - accuracy: 0.3010 - val_loss: 1.5759 - val_accuracy: 0.2818
2008-04-30 00:00:00 2018-04-30 00: